# Drought Classification Workflow

This notebook demonstrates the drought classification workflow for the Snow Drought Index package. It covers loading SSWEI data, classifying drought conditions, analyzing drought characteristics, and visualizing drought patterns over time.

In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import snowdroughtindex package
from snowdroughtindex.core import data_preparation, sswei, drought_classification
from snowdroughtindex.utils import visualization, io

## 1. Load SSWEI Data

First, we'll load the SSWEI data that was calculated in the SSWEI calculation workflow.

In [ ]:
# Load SSWEI data from CSV file
sswei_data = pd.read_csv('../data/processed/sswei_results.csv')

# Display the first few rows of the data
sswei_data.head()

## 2. Classify Drought Conditions with Configurable Thresholds

The drought classification module allows for configurable thresholds. Let's demonstrate how to use custom thresholds for drought classification.

In [ ]:
# Define custom thresholds
custom_thresholds = {
    "exceptional": -1.8,  # More severe threshold for exceptional drought
    "extreme": -1.3,      # More severe threshold for extreme drought
    "severe": -0.8,       # More severe threshold for severe drought
    "moderate": -0.4,     # Less severe threshold for moderate drought
    "normal_lower": -0.4, # Lower bound for normal conditions
    "normal_upper": 0.4,  # Upper bound for normal conditions
    "abnormally_wet": 0.8, # Threshold for abnormally wet conditions
    "moderately_wet": 1.3, # Threshold for moderately wet conditions
    "very_wet": 1.8       # Threshold for very wet conditions
}

# Apply custom classification
sswei_data['Custom_Classification'] = sswei_data['SWEI'].apply(
    lambda x: drought_classification.classify_drought(x, custom_thresholds)
)

# Compare default and custom classifications
comparison = sswei_data[['season_year', 'SWEI', 'Drought_Classification', 'Custom_Classification']]
comparison.head(10)

## 3. Visualize Drought Classifications

Let's visualize the drought classifications using the plotting functions from the drought classification module.

In [ ]:
# Plot default drought classification
fig1 = drought_classification.plot_drought_classification(
    sswei_data,
    year_column='season_year',
    swei_column='SWEI',
    classification_column='Drought_Classification'
)
plt.title('Default Drought Classification')
plt.show()

# Plot custom drought classification
fig2 = drought_classification.plot_drought_classification(
    sswei_data,
    year_column='season_year',
    swei_column='SWEI',
    classification_column='Custom_Classification'
)
plt.title('Custom Drought Classification')
plt.show()

## 4. Calculate Drought Characteristics

Now we'll calculate drought characteristics such as duration, severity, and intensity for each drought event.

In [ ]:
# Calculate drought characteristics using default threshold (-0.5)
drought_chars = drought_classification.calculate_drought_characteristics(
    sswei_data,
    year_column='season_year',
    swei_column='SWEI'
)

# Display drought characteristics
if not drought_chars.empty:
    print("Drought Characteristics:")
    display(drought_chars)
else:
    print("No drought events found in the data.")

## 5. Visualize Drought Characteristics

Let's visualize the drought characteristics to better understand the drought events.

In [ ]:
# Plot drought characteristics
if not drought_chars.empty:
    fig3 = drought_classification.plot_drought_characteristics(drought_chars)
    plt.show()
else:
    print("No drought events to plot.")

## 6. Analyze Drought Trends

Let's analyze drought trends over time using a moving window approach.

In [ ]:
# Define window size for trend analysis
window_size = 10  # 10-year moving window

# Analyze drought trends
trend_data = drought_classification.analyze_drought_trends(
    sswei_data,
    year_column='season_year',
    swei_column='SWEI',
    window_size=window_size
)

# Display trend data
if not trend_data.empty:
    print(f"Drought Trends (using {window_size}-year moving window):")
    display(trend_data.head())
else:
    print("Not enough data for trend analysis.")

## 7. Visualize Drought Trends

Let's visualize the drought trends to identify any changes in drought frequency and severity over time.

In [ ]:
# Plot drought trends
if not trend_data.empty:
    fig4 = drought_classification.plot_drought_trends(trend_data)
    plt.show()
else:
    print("Not enough data for trend visualization.")

## 8. Compare Drought Classifications by Decade

Let's analyze how drought classifications have changed by decade.

In [ ]:
# Add decade column
sswei_data['decade'] = (sswei_data['season_year'] // 10) * 10

# Count classifications by decade
decade_counts = pd.crosstab(sswei_data['decade'], sswei_data['Drought_Classification'])

# Display counts
print("Drought Classifications by Decade:")
display(decade_counts)

# Plot heatmap of classifications by decade
plt.figure(figsize=(12, 8))
sns.heatmap(decade_counts, cmap='YlOrRd', annot=True, fmt='d', cbar_kws={'label': 'Count'})
plt.title('Drought Classifications by Decade')
plt.ylabel('Decade')
plt.xlabel('Drought Classification')
plt.tight_layout()
plt.show()

## 9. Analyze Drought Severity Distribution

Let's analyze the distribution of drought severity values.

In [ ]:
# Calculate drought severity for all years
sswei_data['drought_severity'] = sswei_data['SWEI'].apply(drought_classification.get_drought_severity)

# Plot histogram of drought severity
plt.figure(figsize=(10, 6))
sns.histplot(sswei_data[sswei_data['drought_severity'] > 0]['drought_severity'], bins=10, kde=True)
plt.title('Distribution of Drought Severity')
plt.xlabel('Drought Severity')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate summary statistics for drought severity
severity_stats = sswei_data[sswei_data['drought_severity'] > 0]['drought_severity'].describe()
print("Drought Severity Statistics:")
print(severity_stats)

## 10. Save Drought Analysis Results

Finally, let's save the drought analysis results for future reference.

In [ ]:
# Save drought characteristics
if not drought_chars.empty:
    drought_chars.to_csv('../data/processed/drought_characteristics.csv', index=False)
    print("Drought characteristics saved to '../data/processed/drought_characteristics.csv'")

# Save drought trends
if not trend_data.empty:
    trend_data.to_csv('../data/processed/drought_trends.csv', index=False)
    print("Drought trends saved to '../data/processed/drought_trends.csv'")

# Save SSWEI data with custom classification and severity
sswei_data.to_csv('../data/processed/sswei_with_drought_analysis.csv', index=False)
print("SSWEI data with drought analysis saved to '../data/processed/sswei_with_drought_analysis.csv'")

## 11. Summary

In this notebook, we've demonstrated the drought classification workflow for the Snow Drought Index package. We've loaded SSWEI data, classified drought conditions using both default and custom thresholds, analyzed drought characteristics, visualized drought patterns, and examined drought trends over time.

The workflow uses the following key functions from the `drought_classification` module:
- `classify_drought()` for classifying drought conditions based on SSWEI values
- `calculate_drought_characteristics()` for analyzing drought events
- `analyze_drought_trends()` for examining changes in drought patterns over time
- Various plotting functions for visualizing drought classifications and characteristics

These functions provide a comprehensive toolkit for analyzing drought conditions based on SSWEI data, enabling researchers to better understand drought patterns, severity, and trends over time.